In [1]:
import numpy as np
import scipy as sp
import scipy.linalg
# import pandas as pd

## Basic function definitions

In [2]:
Id = np.eye(2)
X = np.array([[0.0, 1.0],[1.0, 0.0]])
Z = np.array([[1.0, 0.0],[0.0, -1.0]])
Y = np.matmul(X,Z)

zero = np.array([[1.0], [0.0]]) # |0>
one = np.array([[0.0], [1.0]]) # |1>

In [3]:
def NKron(*args):
  result = np.array([[1.0]])
  for op in args:
    result = np.kron(result, op)
  return result

def NormalizeState(ipVal):
    if(sp.linalg.norm(ipVal) == 0): return ipVal
    else : return ipVal / sp.linalg.norm(ipVal)

In [4]:
def NKronModified(checkRowMod):
  result = np.array([[1.0]])
  for ind in checkRowMod:
    if(ind == 0):
        op = Id
    elif(ind == 1):
        op = X
    elif(ind == 2):
        op = Y
    elif(ind == 3):
        op = Z
    result = np.kron(result, op)
  return result

def getGenerator(checkRow):
    checkRowModified = np.zeros(n, dtype=int)
    
    checkRowModified[(checkRow[:n] == checkRow[n:]) & (checkRow[n:] == 1)] = 2
    checkRowModified[(checkRow[:n] == 1) & (checkRowModified != 2)] = 1
    checkRowModified[(checkRow[n:] == 1) & (checkRowModified != 2)] = 3
    
    return NKronModified(checkRowModified)    

## Check matrix

In [5]:
tot_samples = 1000
p_xyz = 0.15

checkMatrix = np.array([[0,0,0,1,1,1,1, 0,0,0,0,0,0,0],
                        [0,1,1,0,0,1,1, 0,0,0,0,0,0,0],
                        [1,0,1,0,1,0,1, 0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0, 0,0,0,1,1,1,1],
                        [0,0,0,0,0,0,0, 0,1,1,0,0,1,1],
                        [0,0,0,0,0,0,0, 1,0,1,0,1,0,1]])

n = int(checkMatrix.shape[1]/2)
k = n-checkMatrix.shape[0]

gi = np.zeros([n-k, 2**n, 2**n])
for i in range(n-k):
    gi[i,:,:] = getGenerator(checkMatrix[i,:])

## G matrix

In [6]:
Gmatrix = np.eye(gi[0,:,:].shape[0], gi[0,:,:].shape[1]) # generator matrix corresponding to this code
for i in range(n-k):
    Gmatrix = Gmatrix + np.matmul(gi[i,:,:], Gmatrix)
Gmatrix = np.round(Gmatrix)

## Non-zero unique columns

In [7]:
# get boolean array if the columns are zero or not
zeroCols = np.zeros(Gmatrix.shape[1])
for i in range(Gmatrix.shape[1]):
    zeroCols[i] = all(Gmatrix[:,i] == np.zeros(Gmatrix.shape[0]))

# get indices of non-zero columns
nonZeroColsList = np.argwhere(zeroCols==0).flatten()

# get all non zero columns
GmatrixNonZero = np.zeros([Gmatrix.shape[0], nonZeroColsList.shape[0]])
i = 0
for ind in nonZeroColsList:
    GmatrixNonZero[:,i] = Gmatrix[:,ind]
    i = i+1

# get all non zero and unique columns and there indices
GmatrixNonZeroUniqueInd, nonZeroUniqueInd = np.unique(GmatrixNonZero, axis = 1, return_index=True)
nonZeroUniqueInd = nonZeroColsList[nonZeroUniqueInd]

## Encoder

In [8]:
def encode_qbits(qbits):
    # get extended qbits corresponding to non-zero column indices of G matrix
    encoded = np.zeros(2**n)
    i = 0
    for nonZeroIndex in np.sort(nonZeroUniqueInd):
        if(i>=2**k):
            break
        encoded[nonZeroIndex] = qbits[i]
        i = i+1
    encoded = NormalizeState(encoded)

    # encode transmit qbits using generators
    for i in range(n-k):
        encoded = encoded + np.matmul(gi[i,:,:], encoded)
    encoded = NormalizeState(encoded)
    
    return encoded

## Channel

In [9]:
def depolarizing_channel(input_qbits):
    p_channel = [1-3*p_xyz, p_xyz, p_xyz, p_xyz] 
    errMatrix = np.random.multinomial(1, p_channel, size=n)
    error_vector = errMatrix@np.array([0,1,2,3])
    channel_error = NKronModified(error_vector)
    
    output_qbits = np.dot(channel_error, input_qbits)
    
    return output_qbits, error_vector.flatten()

## Syndrome check

In [10]:
def get_syndrome(input_qbits):
    syndr = np.zeros(n-k)
    for i in range(n-k):
        syndr[i] = np.dot(input_qbits.transpose(), np.dot(gi[i,:,:], input_qbits))
    syndr = syndr.flatten() 
    
    return syndr

## Generate data

In [11]:
syndrome_col = np.zeros([tot_samples, n-k])
error_col = np.zeros([tot_samples, n])

for i_sample in range(tot_samples):
    
    # generate qbits randomly
    tx_qbits = np.random.rand(2**k)
    tx_qbits = NormalizeState(tx_qbits)
    
    # encode qbits
    tx_encoded = encode_qbits(tx_qbits)
    
    # channel
    rx_erry, error_vector = depolarizing_channel(tx_encoded)
    
    # syndrome
    syndr = get_syndrome(rx_erry)
    
    # fill columns
    syndrome_col[i_sample, :] = syndr
    error_col[i_sample, :] = error_vector

In [13]:
savefile_name = 'data_' + str(n) + ',' + str(k) + '_tot_samples_'+ str(tot_samples) + '_p_' + str(p_xyz)
savefile_name_syndr = savefile_name + '_syndr.csv'
savefile_name_error = savefile_name + '_error.csv'

np.savetxt(savefile_name_syndr, syndrome_col, delimiter=",")
np.savetxt(savefile_name_error, error_col, delimiter=",")

print('Saved files:')
print(savefile_name_syndr)
print(savefile_name_error)

Saved files:
data_7,1_tot_samples_1000_p_0.15_syndr.csv
data_7,1_tot_samples_1000_p_0.15_error.csv
